In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]")\
.config("spark.jars.packages","mysql:mysql-connector-java:5.1.6,org.xerial:sqlite-jdbc:3.23.1,org.mongodb.spark:mongo-spark-connector_2.11:2.3.0,com.datastax.spark:spark-cassandra-connector_2.11:2.3.1")\
.appName("ch09DataSources_1").getOrCreate()

In [3]:
# import pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("yarn")\
# .config("spark.jars","/home/kranthidr/softwares/jars/mysql-connector-java-5.1.6.jar")\
# .appName("ch09DataSources_1").getOrCreate()


# "org.xerial:sqlite-jdbc:3.23.1"
# "org.mongodb.spark:mongo-spark-connector_2.11:2.3.0"
# "com.datastax.spark:spark-cassandra-connector_2.11:2.3.1"


In [4]:
sc = spark.sparkContext

In [5]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

In [6]:
# COMMAND ----------
mysql_url = "jdbc:mysql://nn01.itversity.com/retail_db"
mysql_driver = "com.mysql.jdbc.Driver"
mysql_dbtable = "categories"
mysql_user = "retail_dba"
mysql_password = "itversity"

In [7]:
dataframe_mysql = spark.read.format("jdbc")\
.option("url", mysql_url)\
.option("dbtable",mysql_dbtable)\
.option("driver",mysql_driver)\
.option("user", mysql_user)\
.option("password", mysql_password).load()

In [8]:
dataframe_mysql.show()

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
|          6|                     2|   Tennis & Racquet|
|          7|                     2|             Hockey|
|          8|                     2|        More Sports|
|          9|                     3|   Cardio Equipment|
|         10|                     3|  Strength Training|
|         11|                     3|Fitness Accessories|
|         12|                     3|       Boxing & MMA|
|         13|                     3|        Electronics|
|         14|                     3|     Yoga & Pilates|
|         15|                  

In [9]:
# COMMAND ----------
driver = "org.sqlite.JDBC"
path = "my-sqlite.db"
url = "jdbc:sqlite:" + path
tablename = "flight_info"
#takes local path

In [10]:
# # # COMMAND ----------

dbDataFrame = spark.read.format("jdbc").option("url", url)\
  .option("dbtable", tablename).option("driver",driver).load()
    
    
dbDataFrame.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [11]:
dbDataFrame.filter("DEST_COUNTRY_NAME in ('Anguilla', 'Sweden')").explain()

# # # IllegalArgumentException: u"requirement failed: Option 'dbtable' is required."

== Physical Plan ==
*(1) Scan JDBCRelation(flight_info) [numPartitions=1] [DEST_COUNTRY_NAME#17,ORIGIN_COUNTRY_NAME#18,count#19] PushedFilters: [*In(DEST_COUNTRY_NAME, [Anguilla,Sweden])], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:decimal(20,0)>


In [12]:
url

'jdbc:sqlite:my-sqlite.db'

In [13]:
pwd

u'/home/kranthidr/projects/ml_lab/pyspark-lab/spark-tdg'

In [14]:
# COMMAND ----------
# pgDF = spark.read.format("jdbc")\
#   .option("driver", "org.postgresql.Driver")\
#   .option("url", "jdbc:postgresql://database_server")\
#   .option("dbtable", "schema.tablename")\
#   .option("user", "username").option("password", "my-secret-password").load()

In [15]:
# COMMAND ----------

pushdownQuery = """(SELECT DISTINCT(DEST_COUNTRY_NAME) FROM flight_info)
  AS flight_info"""

In [16]:
dbDataFrame = spark.read.format("jdbc")\
  .option("url", url).option("dbtable", pushdownQuery).option("driver",  driver)\
  .load()

In [17]:
dbDataFrame.show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|       United States|
|               Egypt|
|   Equatorial Guinea|
|          Costa Rica|
|             Senegal|
|              Guyana|
|               Malta|
|             Bolivia|
|            Anguilla|
|Turks and Caicos ...|
|Saint Vincent and...|
|               Italy|
|            Pakistan|
|             Iceland|
|    Marshall Islands|
|          Luxembourg|
|            Honduras|
|         The Bahamas|
|         El Salvador|
|               Samoa|
+--------------------+
only showing top 20 rows



In [18]:
# COMMAND ----------
dbDataFrame = spark.read.format("jdbc")\
  .option("url", url).option("dbtable", tablename).option("driver",  driver)\
  .option("numPartitions", 10).load()

In [19]:
dbDataFrame.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [23]:
# COMMAND ----------
props = {"driver":"org.sqlite.JDBC"}
predicates = [
  "DEST_COUNTRY_NAME = 'India' OR ORIGIN_COUNTRY_NAME = 'India'",
  "DEST_COUNTRY_NAME = 'Sweden' OR ORIGIN_COUNTRY_NAME = 'Sweden'",
  "DEST_COUNTRY_NAME = 'Anguilla' OR ORIGIN_COUNTRY_NAME = 'Anguilla'"]

spark.read.jdbc(url, tablename, predicates=predicates, properties=props).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|              India|   69|
|            India|      United States|   66|
|           Sweden|      United States|   65|
|    United States|             Sweden|   73|
|         Anguilla|      United States|   21|
|    United States|           Anguilla|   20|
+-----------------+-------------------+-----+



In [24]:
spark.read.jdbc(url,tablename,predicates=predicates,properties=props).rdd.getNumPartitions() # 2

3

In [ ]:
# COMMAND ----------

props = {"driver":"org.sqlite.JDBC"}
predicates = [
  "DEST_COUNTRY_NAME != 'Sweden' OR ORIGIN_COUNTRY_NAME != 'Sweden'",
  "DEST_COUNTRY_NAME != 'Anguilla' OR ORIGIN_COUNTRY_NAME != 'Anguilla'"]
spark.read.jdbc(url, tablename, predicates=predicates, properties=props).count()

In [25]:
# COMMAND ----------
colName = "count"
lowerBound = 0L
upperBound = 348113L # this is the max count in our database
numPartitions = 10

In [26]:
# COMMAND ----------
partitionsDF = spark.read.jdbc(url, tablename, column=colName, properties=props,
                lowerBound=lowerBound, upperBound=upperBound,
                numPartitions=numPartitions)

In [27]:
partitionsDF.count()

255

In [28]:
partitionsDF.rdd.getNumPartitions()

10

In [29]:
csvFile = spark.read.format("csv")\
  .option("header", "true")\
  .option("mode", "FAILFAST")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/dataSets/spark-guide/flight-data/csv/2010-summary.csv")

In [30]:
csvFile.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [31]:
# COMMAND ----------
props = {"driver":"org.sqlite.JDBC"}
newPath = "jdbc:sqlite://tmp/my-sqlite.db" #It is Local Path
csvFile.write.jdbc(newPath, "flight_info", mode="overwrite", properties=props)

In [32]:
# COMMAND ----------
spark.read.jdbc(newPath, tablename, properties=props).count() # 255

255

In [33]:
# COMMAND ----------
csvFile.write.jdbc(newPath, tablename, mode="append", properties=props)

In [34]:
# COMMAND ----------
spark.read.jdbc(newPath, tablename, properties=props).count()

510

In [40]:
textDF = spark.read.text("/user/kranthidr/dataSets/spark-guide/flight-data/csv/2010-summary.csv")

In [44]:
textDF.show(5, False)

+-------------------------------------------+
|value                                      |
+-------------------------------------------+
|DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count|
|United States,Romania,1                    |
|United States,Ireland,264                  |
|United States,India,69                     |
|Egypt,United States,24                     |
+-------------------------------------------+
only showing top 5 rows



In [43]:
textDF.selectExpr("split(value, ',') as rows").show(5, False)

+-----------------------------------------------+
|rows                                           |
+-----------------------------------------------+
|[DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count]|
|[United States, Romania, 1]                    |
|[United States, Ireland, 264]                  |
|[United States, India, 69]                     |
|[Egypt, United States, 24]                     |
+-----------------------------------------------+
only showing top 5 rows



In [38]:
# COMMAND ----------
csvFile.limit(10).select("DEST_COUNTRY_NAME", "count")\
  .write.partitionBy("count").text("/tmp/five-csv-files2py.csv")

In [39]:
# COMMAND ----------
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")\
  .save("/tmp/partitioned-files.parquet")

In [45]:
csvFile.repartition(5).write.format("csv").save("/user/kranthidr/temp/multiple.csv")

In [57]:
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")\
.format("csv").save("/user/kranthidr/temp/partitionBy.csv")

In [58]:
csvFile.selectExpr("DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME as same").groupBy("same").count().show()

+-----+-----+
| same|count|
+-----+-----+
| true|    1|
|false|  254|
+-----+-----+



In [64]:
numBuckets = 10
columnToBucketBy = "count"

In [63]:
# csvFile.write.format("parquet").mode("overwrite").bucketBy(numBuckets, columnToBucketBy)\
# .saveASTable("/user/kranthidr/temp/bucket.parquet")

#This workd only for Spark Managed Tables

In [69]:
# COMMAND ----------
spark.conf.set("spark.cassandra.connection.host", "wn05.itversity.com")

In [70]:
from pyspark.sql import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [78]:
spark.read.format("org.apache.spark.sql.cassandra").options(table="orders", keyspace="raj").load().show()

+--------+-----------------+--------------------+---------------+
|order_id|order_customer_id|          order_date|   order_status|
+--------+-----------------+--------------------+---------------+
|   12300|              127|2013-10-08 00:00:...|PENDING_PAYMENT|
|   52271|             3332|2014-06-18 00:00:...|         CLOSED|
|   36223|             1020|2014-03-04 00:00:...|        PENDING|
|   46440|            11945|2014-05-09 00:00:...|       COMPLETE|
|   16058|             9182|2013-11-03 00:00:...|         CLOSED|
|   48208|             9723|2014-05-20 00:00:...|     PROCESSING|
|   67948|             2071|2013-12-26 00:00:...|         CLOSED|
|   30697|             8412|2014-01-31 00:00:...|        ON_HOLD|
|    4016|            11946|2013-08-17 00:00:...|PENDING_PAYMENT|
|   68247|             2435|2014-03-19 00:00:...|         CLOSED|
|   52486|             7946|2014-06-19 00:00:...|       COMPLETE|
|    1006|             9028|2013-07-30 00:00:...|       COMPLETE|
|   16751|